In [1]:
import numpy as np
import pandas as pd

import sys
print(f"Python version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

Python version: 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]
pandas version: 1.4.2
numpy version: 1.22.3


In [2]:
data = pd.read_csv("data/EXP1.csv")
data.head()

,Date,Open,High,Low,Close,Volume
0,2001-01-01,37.25,38.25,35.75,37.50,1234000
1,2001-01-02,39.50,40.12,39.25,40.00,1567000
2,2001-01-03,37.50,38.50,37.00,37.25,1456000
3,2001-01-04,37.00,37.50,36.50,37.00,1789000
4,2001-01-05,37.00,40.25,37.00,39.00,2345000


In [3]:
data.describe()

,Open,High,Low,Close,Volume
count,5.000000,5.000000,5.000000,5.000000,5.000000e+00
mean,37.650000,38.924000,37.100000,38.150000,1.678200e+06
std,1.054751,1.209392,1.306235,1.294218,4.230694e+05
min,37.000000,37.500000,35.750000,37.000000,1.234000e+06
25%,37.000000,38.250000,36.500000,37.250000,1.456000e+06
50%,37.250000,38.500000,37.000000,37.500000,1.567000e+06
75%,37.500000,40.120000,37.000000,39.000000,1.789000e+06
max,39.500000,40.250000,39.250000,40.000000,2.345000e+06


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    5 non-null      object 
 1   Open    5 non-null      float64
 2   High    5 non-null      float64
 3   Low     5 non-null      float64
 4   Close   5 non-null      float64
 5   Volume  5 non-null      int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 368.0+ bytes


In [5]:
price_data = data[['High','Low','Close']]

In [6]:
price_data.head()

,High,Low,Close
0,38.25,35.75,37.50
1,40.12,39.25,40.00
2,38.50,37.00,37.25
3,37.50,36.50,37.00
4,40.25,37.00,39.00


Two methods for populating charts:

- **High and Low** price: whenever H and L are available
- **Last/Close** price: for illiquid assets (or indices). Also for intraday trading using realtime data

Scale types:

- Constant size boxes
- Variable size boxes

In [8]:
def init_pnf(scale,
             high,
             low,
             close,
             reversal_size,
             box_range=[]):
    '''
    returns status as num value and box_range as array
    '''
    
    if len(box_range) == 0:
        box_range = scale[np.logical_and(scale>=low, scale<=high)]
    else:
        # simplify logic using max()
        if high > box_range.max():
            box_range = scale[np.logical_and(scale>=box_range.min(), scale<=high)]
        if low < box_range.min():
            box_range = scale[np.logical_and(scale<=box_range.max(), scale>=low)]
        
    # check definition of mid_price
    mid_price = box_range.min() + (box_range.max() - box_range.min())/2

    if len(box_range) >= reversal_size and close > mid_price:
        status = 1
    elif len(box_range) >= reversal_size and close < mid_price:
        status = -1
    else:
        status = 0
        
    return status, box_range

In [10]:
# checking init_pnf() on the 1st line of data:

box_size = 1
reversal_size = 3
scale = np.arange(start=30, stop=49, step = box_size)

high = 38.75
low = 35.75
close = 37.00

status, box_range = init_pnf(scale, high, low, close, reversal_size)

In [11]:
box_range

array([36, 37, 38])

In [12]:
scale

array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
       47, 48])

In [59]:
# checking init_pnf() on subsequent lines of data (when status==0)

high = 39.25
low = 35.01
close = 38

init_pnf(scale, high, low, close, reversal_size, box_range)

(1, array([36, 37, 38, 39]))

In [60]:
# initialise status and box arrays:
trend_status = np.zeros(len(price_data))
box_low = np.zeros(len(price_data))
box_high = np.zeros(len(price_data))

In [62]:
# Initialise the chart until a status (+/-)1 is reached
for index, row in enumerate(price_data.iterrows()):
    high = row[1]['High']
    low = row[1]['Low']
    close = row[1]['Close']
    status, box_range = init_pnf(scale, high, low, close, reversal_size, box_range)
    trend_status[index] = status
    box_low[index] = box_range.min()
    box_high[index] = box_range.max()
    if status != 0:
        break

# status can still be 0! create an example for testing
print(index, trend_status, box_low, box_high)

0 [1. 0. 0. 0. 0.] [36.  0.  0.  0.  0.] [38.  0.  0.  0.  0.]


To obtain the range again given min and max:

In [22]:
scale[np.logical_and(scale>=36, scale<=38)]

array([36, 37, 38])

In [64]:
# testing reversal logic

box_range = np.array([36, 37, 38])
status = 1
scale = scale
reversal_size = 3

high = 38.5
low = 35.5
box_low = box_range.min()
box_high = box_range.max()

box_reverse = []
# this can be simplified into one line using max()
# new range with extensions on both sides:
if high > box_high:
    box_range_new = scale[np.logical_and(scale>=box_low, scale<=high)]
if low < box_range.min():
    box_range_new = scale[np.logical_and(scale<=box_high, scale>=low)]

# we actually need only box_high_new and box_low_new
# box_high_new = box_range_new.max()
# box_low_new = box_range_new.min()

if status == 1:
    # we 1st check for extensions on the upside
    if box_range_new.max() > box_high:
        # extend range up:
        box_range = scale[np.logical_and(scale>=box_low, scale<=box_range_new.max())]
    # if no extension, check for reversal:
    elif low < box_high:
        box_reverse = scale[np.logical_and(scale>=low, scale<=box_high)][:-1]

if status == -1:
    # we 1st check for extensions on the downside
    if box_range_new.min() < box_low:
        # extend range down:
        box_range = scale[np.logical_and(scale>=box_range_new.min(), scale<=box_high)]
    # if no extension, check for reversal:
    elif high > box_low:
        box_reverse = scale[np.logical_and(scale>=box_low, scale<=high)][1:]

if len(box_reverse) >= reversal_size:
    status *= -1 # reverse status
#     box_range = box_reverse
    
print(box_range, box_range_new, box_reverse, status)

[36 37 38] [36 37 38] [36 37] 1


We then implement the logic into a update_pnf() function:

In [14]:
def update_pnf(scale,
               high,
               low,
               status,
               reversal_size,
               box_low,
               box_high):
    '''
    updates the chart once the trend status is defined
    returns status and box_range for the day
    '''
    box_range = scale[np.logical_and(scale>=box_low, scale<=box_high)] # needed in case we return the previous range
    box_reverse = []
    # new temporary box range with extensions on both sides:
    
    if status == 1:
        # check for upper extensions, else for reversals

            # check for reversal
            # research condition
            # change status and new range
#             box_range_temp
                
    
    if status == -1:

            #check for reversal
            #update status
#             box_range_temp
            
    if len(box_range_temp) >= reversal_size:
        status *= -1 # reverse status
        box_range = box_range_temp # update box_range
            
    return status, box_range

IndentationError: expected an indented block (270733341.py, line 7)

Below we want to run the chart functions on the available data:

In [63]:
# check there are more lines of data
index + 1 < len(price_data)

True

In [ ]:
status = 0
box_range = []
for row in price_data.iterrows():
    high = row[1]['High']
    low = row[1]['Low']
    close = row[1]['Close']
    status, box_range = init_pnf(scale, high, low, close, reversal_size, box_range)
    if status != 0:
        print(row[0], status, box_range)
        break

In [57]:
def chart_grid(scale, columns):
    '''
    Generates a blank monospace-font grid that can be used as a canvas for the PnF chart
    - scale: a np array
    - columns: a positive integer
    '''
    columns = int(columns)
    
    grid = ""

    for level in np.flip(scale):
        line_price = level
        line = f'{line_price}' + '.'*columns + f'{line_price}\n'
        grid += line
    return grid

print(chart_grid(scale, 25))

48.........................48
47.........................47
46.........................46
45.........................45
44.........................44
43.........................43
42.........................42
41.........................41
40.........................40
39.........................39
38.........................38
37.........................37
36.........................36
35.........................35
34.........................34
33.........................33
32.........................32
31.........................31
30.........................30

